<a href="https://colab.research.google.com/github/sebasruggero/delitosCabaH3/blob/main/Delitos_Caba_2016_2021_H3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install h3
!pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.2 MB/s eta 0:00:00


In [2]:
# Importamos las bibliotecas necesarias para hacer el proceso de transformacion .. el archivo fue procesado previamiente con datos del 2016 al 2021
import pandas as pd

In [7]:
# Definimos el path en este caso nos contectamos al drive // Enlace al csv (https://drive.google.com/file/d/1YUMD_j2rUItGeg4vVrZPZ8hdZ87K5UfI/view?usp=sharing)
path = '/content/drive/Othercomputers/My Laptop/DataScience/Data/DataFrames/delitos_totales_2016_2021.csv'

In [8]:
# Levantamos el csv desde el drive con pandas
df = pd.read_csv(path, sep= "\t" , on_bad_lines='warn')

<ipython-input-8-47e9512365fd>:2: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep= "\t" , on_bad_lines='warn')


In [9]:
# Verificamos los tipos de datos
print(df.dtypes)

Unnamed: 0               int64
fecha                   object
franja_horaria          object
tipo_delito             object
subtipo_delito          object
cantidad_registrada    float64
comuna                 float64
barrio                  object
lat                     object
long                    object
uso_armas               object
dtype: object


In [10]:
# Función para limpiar valores no numéricos
def clean_numeric(value):
    try:
        return float(value)
    except ValueError:
        return None  # Otra opción podría ser reemplazar con un valor predeterminado

# Aplicar la función de limpieza a las columnas
df['lat'] = df['lat'].apply(clean_numeric)
df['long'] = df['long'].apply(clean_numeric)

# Eliminar filas con valores no numéricos si es necesario
df = df.dropna(subset=['lat', 'long'])

# Ahora, 'lat' y 'long' deberían contener números de punto flotante
print(df)

        Unnamed: 0       fecha franja_horaria            tipo_delito  \
0                0  2016-01-01             13   Robo (con violencia)   
1                1  2016-01-01             22   Robo (con violencia)   
2                2  2016-01-01              1               Lesiones   
3                3  2016-01-01             19  Hurto (sin violencia)   
4                4  2016-01-01              4   Robo (con violencia)   
...            ...         ...            ...                    ...   
560084       66720  2020-12-31              4   Robo (con violencia)   
560085       66721  2020-12-31             16   Robo (con violencia)   
560086       66722  2020-12-31             13   Robo (con violencia)   
560087       66723  2020-12-31              4   Robo (con violencia)   
560088       66724  2020-12-31              9   Robo (con violencia)   

        subtipo_delito  cantidad_registrada  comuna             barrio  \
0                  NaN                  1.0     9.0  Parque A

In [11]:
# Guaradmos el dataset nuevo con las transformaciones
df.to_csv('/content/drive/Othercomputers/My Laptop/DataScience/Data/DataFrames/data_final_delitos.csv')

In [12]:
# Cargamos los datos del dataset en el cual pasamos los valores lat y long a float
data = pd.read_csv('/content/drive/Othercomputers/My Laptop/DataScience/Data/DataFrames/data_final_delitos.csv') # Enlace al archivo csv (https://drive.google.com/file/d/1nxIwbjSOsaOrQN1BIDwUu9ZmToT1Hx0w/view?usp=sharing)

<ipython-input-12-9c760a52f094>:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/Othercomputers/My Laptop/DataScience/Data/DataFrames/data_final_delitos.csv') # Enlace al archivo csv (https://drive.google.com/file/d/1nxIwbjSOsaOrQN1BIDwUu9ZmToT1Hx0w/view?usp=sharing)


In [13]:
# Instalamos los paquetes necesarios para realizar el mapa
! pip install ipyleaflet ipywidgets streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.5 MB/s eta 0:00:00


In [ ]:
# Importamos las librerias necesarias para utilizar h3 , folium ( lo usaremos para realizar el mapa hexagonal ) y los widgets para la interaccion con el mapa.
from h3 import h3
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from ipywidgets import interact, Dropdown

# Define la resolución de los hexágonos H3 (ajusta según sea necesario) -
resolution = 8

# Convierte las coordenadas de latitud y longitud en índices H
data['h3_index'] = data.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'], resolution), axis=1)

# Crea un mapa de colores para la densidad de casos
cmap = cm.YlOrRd

# Define una función para actualizar el mapa según el tipo de delito seleccionado
def update_map(selected_tipo_delito, selected_map_style):
    m = folium.Map(location=[-34.6131500, -58.3772300], zoom_start=11, tiles=selected_map_style)

    # Filtra los datos por el tipo de delito seleccionado
    filtered_data = data[data['tipo_delito'] == selected_tipo_delito]

    # Calcula la densidad de casos por índice H3 para el tipo de delito seleccionado
    density_data = filtered_data['h3_index'].value_counts().reset_index()
    density_data.columns = ['h3_index', 'crime_count']

    # Agrega los hexágonos de densidad al mapa para el tipo de delito seleccionado
    for index, row in density_data.iterrows():
        h3_index = row['h3_index']
        crime_count = row['crime_count']
        lat, long = h3.h3_to_geo(h3_index)

        # Obtiene el color del hexágono según la densidad
        color = mcolors.rgb2hex(cmap(crime_count / density_data['crime_count'].max()))

        folium.Polygon(locations=h3.h3_to_geo_boundary(h3_index),
                       color='white',
                       weight=0,
                       fill=True,
                       fill_color=color,
                       fill_opacity=0.8,
                       popup=f'Casos: {crime_count}').add_to(m)

    return m

# Obtenemos los valores únicos de tipo_delito
tipo_delito_options = list(data['tipo_delito'].unique())

# Creamos un widget de selección para elegir el tipo de delito
tipo_delito_dropdown = Dropdown(options=tipo_delito_options, description='Tipo de Delito')


# Definimos una lista de opciones de estilos de mapa
map_styles = [ "CartoDB dark_matter", "OpenStreetMap", "Stamen Terrain", "Stamen Toner", "Stamen Watercolor", "CartoDB positron"]

# Creamos un widget de selección para elegir el estilo de mapa
map_style_dropdown = Dropdown(options=map_styles, description='Estilo de Mapa')

# Creamos un mapa inicial con el primer estilo de mapa y el primer tipo de delito
initial_map = folium.Map(location=[-34.6131500, -58.3772300], zoom_start=11, tiles=map_styles[0])

# Creamos un widget interactivo para cambiar el tipo de delito y el estilo de mapa
interact = interact(update_map, selected_tipo_delito=tipo_delito_dropdown, selected_map_style=map_style_dropdown)

interactive(children=(Dropdown(description='Tipo de Delito', options=('Robo (con violencia)', 'Lesiones', 'Hur…

In [25]:
# prompt: Using dataframe data: cuenta por tipo de delito ordenado por barrio de mayor a menor y guardalo en un df

data.groupby('barrio')['tipo_delito'].value_counts().sort_values(ascending=False).reset_index(name='count').rename(columns={'tipo_delito':'delito'})


,barrio,delito,count
0,Palermo,Robo (con violencia),22707
1,Balvanera,Robo (con violencia),20075
2,Palermo,Hurto (sin violencia),18400
3,Flores,Robo (con violencia),16932
4,Balvanera,Hurto (sin violencia),15702
...,...,...,...
192,Agronomía,Homicidio,3
193,Parque Chas,Homicidio,2
194,Villa Del Parque,Homicidio,1
195,Coghlan,Homicidio,1
